# Continuized MGs

A sketch of how to continuize MGs.

Intuition: categories are containers for phonological content.

In [2]:
type Phon = String

newtype N = N Phon deriving Eq
newtype D = D Phon deriving Eq
newtype V = V Phon deriving Eq
newtype T = T Phon deriving Eq

instance Show N where
    show (N phon) = unwords [phon,":","n"]
instance Show D where
    show (D phon) = unwords [phon,":","d"]
instance Show V where
    show (V phon) = unwords [phon,":","v"]
instance Show T where
    show (T phon) = unwords [phon,":","t"]

Categorizers wrap phonological content in a category.

In [3]:
n :: Phon -> N
n = N

dog = n "dog"
dog

house = n "house"
house

dog : n

house : n

Complement-taking heads pronounce their complements to their right. This behaviour is built into their category.

In [4]:
d :: Phon -> N -> D
d head (N comp) = D (unwords [head,comp])

v0 :: Phon -> V
v0 = V

v1 :: Phon -> D -> V
v1 head (D comp) = V (unwords [head,comp])

the = d "the"
arrive = v1 "arrive"
rain = v0 "rain"

Merge is just function application.

In [5]:
the dog
arrive (the dog)
rain

the dog : d

arrive the dog : v

rain : v

Specifiers are pronounced to the left -- again, this is built into the behaviour of specifier taking heads.

In [6]:
t :: Phon -> V -> D -> T
t head (V comp) (D spec) = T (unwords [spec,head,comp])

expl :: D
expl = D "it"

past = t "did"

past rain expl

it did rain : t

In order to define move, I'm going to need to use the **indexed continuation applicative**.

In [7]:
newtype IxCont r i a = IxCont { (>>-) :: (a -> i) -> r }

ixMap :: (a -> b) -> IxCont r i a -> IxCont r i b
ixMap f m = IxCont $ \k -> m >>- \x -> k $ f x


ixPure :: a -> IxCont r r a
ixPure a = IxCont $ \k -> k a

ixAp :: IxCont r r' (a -> b) -> IxCont r' r'' a -> IxCont r r'' b
m `ixAp` n = IxCont $ \k -> m >>- \f -> n >>- \x -> k $ f x

ixLower :: IxCont r r' r' -> r
ixLower m = m >>- id

Move as copy

In [12]:
move lex = IxCont $ \k -> k lex lex

ixLower $ ixPure past `ixAp` (ixPure arrive `ixAp` move (the dog))

the dog did arrive the dog : t